In [19]:
import pandas as pd
import numpy as np
import talib as ta
import os
import matplotlib.pyplot as plt
import warnings
import pymongo
from datetime import datetime
warnings.filterwarnings('ignore')

In [20]:
dataCoin = pd.read_csv('../data/dataCoin.csv')
dataCoin.set_index('datetime',inplace=True)
dataCoins = []
dataCoins.append(dataCoin.loc['2018-05-01':'2019-05-01'])
dataCoins.append(dataCoin.loc['2019-05-01':'2020-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-09-23'])
dataCoins.append(dataCoin.loc['2021-05-01':'2021-09-23'])
dataCoins.append(dataCoin.loc[:])

In [21]:
def cal_indicator(dataCoin):    
    dataCoin['pctChange'] = dataCoin['close'].pct_change()
    dataCoin['pctStd'] = ta.STDDEV(dataCoin['pctChange'], 24*5)*(24**0.5)
    dataCoin['pctStdVma'] = ta.SUM(dataCoin['pctStd']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
    dataCoin['closeVma'] = ta.SUM(dataCoin['close']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
#     dataCoin['pctStdVSma'] = ta.EMA(dataCoin.pctStdVma, 24*3)
#     dataCoin['pctStdVLma'] = ta.MA(dataCoin.pctStdVma, 24*10)
for data in dataCoins:
    cal_indicator(data)

In [22]:
def get_stat(df):
    se = df.describe()['pctStd']
    se = se.append(pd.Series(df.skew()['pctStd'],index=['skew']))
    se = se.append(pd.Series(df.kurt()['pctStd'],index=['kurt']))
    se = se.append(pd.Series(se['75%']-se['25%'],index=['75%-25%']))
    return se

In [23]:
ses = []
for data in dataCoins:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2019.5',
              '2019.5 - 2020.5',
              '2020.5 - 2021.5',
              '2020.5 - 2021.9',
              '2021.5 - 2021.9',
              '2018.5 - 2021.9']
df

,2018.5 - 2019.5,2019.5 - 2020.5,2020.5 - 2021.5,2020.5 - 2021.9,2021.5 - 2021.9,2018.5 - 2021.9
count,8606.000000,8635.000000,8625.000000,12077.000000,3332.000000,29558.000000
mean,0.044842,0.042861,0.043741,0.047588,0.057621,0.045311
std,0.019909,0.024184,0.019275,0.021991,0.025500,0.022128
min,0.012783,0.015876,0.012339,0.012339,0.026804,0.012339
25%,0.029456,0.029858,0.030793,0.033405,0.042123,0.031018
50%,0.040496,0.037355,0.038087,0.042010,0.049184,0.040284
75%,0.057324,0.048837,0.053503,0.056531,0.062675,0.054527
max,0.109565,0.202287,0.113070,0.162451,0.162451,0.202287
skew,0.765256,3.449153,1.193070,1.728748,2.123655,2.160471
kurt,0.213007,16.875474,1.167480,4.446964,4.724682,8.604505


In [24]:
data_over_ma = []
data_below_ma = []
dataCoinsMA = []
dataCoinsMA.append(dataCoin.loc['2018-05-01':'2020-05-01'])
dataCoinsMA.append(dataCoin.loc['2020-05-01':'2021-09-23'])
dataCoinsMA.append(dataCoin.loc['2020-12-01':'2021-04-30'])
for data in dataCoinsMA:
    cal_indicator(data)

for data in dataCoinsMA:
    data_over_ma.append(data[data['close'] >= data['closeVma']])
    data_below_ma.append(data[data['close'] < data['closeVma']])

In [25]:
ses = []
for data in data_over_ma:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2020.5',
              '2020.5 - 2021.9',
             '2020.12 - 2021.4']
print('OVER MA')
df

OVER MA


,2018.5 - 2020.5,2020.5 - 2021.9,2020.12 - 2021.4
count,6546.000000,8198.000000,2250.000000
mean,0.042114,0.046677,0.053252
std,0.017615,0.019283,0.017493
min,0.015050,0.012853,0.026479
25%,0.030393,0.033210,0.038064
50%,0.037939,0.041951,0.051042
75%,0.050887,0.055881,0.066435
max,0.109565,0.143626,0.093035
skew,1.242668,1.111208,0.505417
kurt,1.821309,1.198648,-0.918837


In [26]:
ses = []
for data in data_below_ma:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2020.5',
              '2020.5 - 2021.9',
              '2020.12 - 2021.4']
print('BELOW MA')
df

BELOW MA


,2018.5 - 2020.5,2020.5 - 2021.9,2020.12 - 2021.4
count,9736.000000,2800.000000,140.000000
mean,0.044095,0.054764,0.047260
std,0.025490,0.029277,0.015520
min,0.012783,0.012339,0.031632
25%,0.028027,0.036853,0.036468
50%,0.036376,0.045153,0.041695
75%,0.054454,0.063823,0.059636
max,0.202287,0.162451,0.090848
skew,2.647676,1.665073,1.333408
kurt,11.213942,2.983359,0.627791
